In [2]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
import os

In [3]:
import csv
from typing import Tuple

import numpy as np
from qulacs import Observable
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from scikit_quri.circuit.pre_defined import create_dqn_cl, create_dqn_cl_no_cz
from scikit_quri.circuit import LearningCircuit
from scikit_quri.qnn.classifier import QNNClassifier
from quri_parts.core.estimator.gradient import (
    create_numerical_gradient_estimator,
    create_parameter_shift_gradient_estimator)
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_estimator,
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.algo.optimizer import Adam
from quri_parts.core.operator import Operator, pauli_label
# from scikit_quri.qnn.solver import Adam

In [4]:
# Use wine dataset retrieved from: https://archive-beta.ics.uci.edu/ml/datasets/wine
def load_dataset(
    file_path: str, ignore_kind: int, test_ratio: float
) -> Tuple[np.array, np.array, np.array, np.array]:
    """Load dataset from specified path.

    Args:
        file_path: File path from which data is loaded.
        ignore_kind: The dataset expected to have 3 classes and we need 2 classes to test. So specify here which class to ignore in loading.
    """
    x = []
    y = []
    with open(file_path) as f:
        reader = csv.reader(f)
        for row in reader:
            kind = int(row[0])
            if kind == ignore_kind:
                continue
            y.append(kind)
            x.append([float(feature) for feature in row[1:]])

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_ratio, shuffle=True
    )

    return x_train, x_test, y_train, y_test

In [5]:
def create_classifier(n_features: int, circuit:LearningCircuit, locality:int):
    adam = Adam()
    estimator = create_qulacs_vector_concurrent_estimator()
    gradient_estimator = create_numerical_gradient_estimator(create_qulacs_vector_concurrent_parametric_estimator(),delta=1e-3)
    ops = []
    for i in range(n_features):
        if i < locality:
            op = Operator({pauli_label(f"Z {i}"):1.0})
        # else:
        #     op = Operator({pauli_label(f"I {i}"):1.0}) 
        ops.append(op)

    classifier = QNNClassifier(circuit,2,estimator,gradient_estimator,optimizer=adam,operator=ops)
    return classifier


In [8]:
x_train, x_test, y_train, y_test = load_dataset("../datasets/wine.data", 3, 0.5)
for i in range(len(y_train)):
    y_train[i] -= 1
for i in range(len(y_test)):
    y_test[i] -= 1
from icecream import ic

n_features = 13
locality = 2
maxiter = 5

n_train = 10
n_test = 10
x_train = x_train[:n_train]
y_train = y_train[:n_train]
x_test = x_test[:n_test]
y_test = y_test[:n_test]
print("CL")
circuit = create_dqn_cl(n_features, 5, locality)
classifier = create_classifier(n_features,circuit,locality)
result_cl = []
loop_size = 80
print(len(x_train))
for i in range(loop_size):
    classifier.fit(np.array(x_train), np.array(y_train), maxiter)
    ic(len(np.array(x_train)))
    ic(len(np.array(y_train)))
    y_pred = classifier.predict(np.array(x_test))
    score = f1_score(y_test, y_pred, average="weighted")
    result_cl.append(score)
    print(f"iteration:{(i+1)*maxiter} score:{score}")

print("no CL")
circuit = create_dqn_cl_no_cz(n_features, 5)
classifier = create_classifier(n_features,circuit, locality)
result_no_cl = []
for i in range(loop_size):
    classifier.fit(np.array(x_train),np.array(y_train), maxiter)
    y_pred = classifier.predict(np.array(x_test))
    score = f1_score(y_test, y_pred, average="weighted")
    result_no_cl.append(score)
    print(f"iteration:{(i+1)*maxiter} score:{score}")

CL
10
init_params=array([1.74025881, 0.5178033 , 4.83410921, 4.01984367, 5.81631226,
       5.75207854, 2.27310803, 4.74576703, 3.80821522, 4.61882777,
       2.15744856, 2.9098306 , 2.09079194, 3.5583184 , 1.36951114,
       4.52022703, 0.11908529, 2.15793602, 5.41990056, 5.33644722,
       3.85589898, 1.3172547 , 4.43377615, 6.12914039, 3.30201607,
       0.9604737 , 0.38940565, 6.22934812, 0.55298106, 4.01707543,
       2.18770213, 2.8436079 , 4.27010389, 5.06308856, 3.58069263,
       4.73446435, 3.93703973, 1.60310091, 1.83532288, 2.53407617,
       4.14081338, 0.13543379, 4.58102302, 0.51145438, 4.56420368,
       3.27363623, 0.32627426, 3.72967403, 2.3948243 , 5.33282482,
       0.31746767, 0.38567242, 3.24770899, 3.20862517, 4.87141631,
       1.5415098 , 3.95453805, 4.23805482, 2.2430176 , 3.47134514,
       1.52913459, 0.81508237, 1.9270812 , 4.56714436, 4.01192564,
       1.27937881, 6.13436921, 1.7943491 , 0.83546674, 4.83852057,
       5.68244382, 0.04636684, 5.17712531, 2

ic| len(np.array(x_train)): 10
ic| len(np.array(y_train)): 10


 pred_inner:1/2

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.arange(0, loop_size*maxiter, step=maxiter), np.array(result_cl).flatten(), label="CL")
plt.plot(np.arange(0, loop_size*maxiter, step=maxiter), np.array(result_no_cl).flatten(), label="no CL")
plt.xticks(np.arange(0, loop_size*maxiter, step=25))
plt.xlabel("iteration")
plt.ylabel("score")
plt.legend()
plt.show()